<a href="https://colab.research.google.com/github/IRPARKS/NMML/blob/main/NMML_PROJECT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rat08-20130711_017.h5

In [ ]:
import h5py
import numpy as np

filepath = 'Rat08-20130711_017.h5'  # data file
f = h5py.File(filepath, 'r')  # read data with h5 format
fs = f.attrs['fs'][0]  # get sampling frequency of LFP signal (Hz)
print("Sampling rate: %.1f Hz" % (fs))

In [ ]:
states = []  # two states (NREM & WAKE) to be classified
# LFP recordings are store in two h5 groups for each state
# Under each h5 group, the LFP recordings are divided into several segments with different lengths.
for name, grp in f.items():
  states.append(name)
  print("State: %s" % (name))
  print("Segment IDs:")
  print(list(grp.keys()))

In [ ]:
# Convert the recording in to numpy arrays
# Use a dictionary to store the LFP recordings of the two states
# each containing a list of numpy arrays of all segments
lfp = {key: [] for key in states}
for key in states:
  group = f[key]  # h5 group of a state
  n = len(group)  # number of segments
  for i in range(n):
    lfp[key].append(group[str(i+1)][()].astype(float))  # convert data to numpy array and from int type to float type

# print(lfp)

In [ ]:
import matplotlib.pyplot as plt

x = lfp['NREM'][10]  # accessing the 10-th LFP segment in NREM state
t = np.arange(x.size)/fs  # time points

plt.plot(t,x)
plt.xlabel('second')
plt.show()

In [ ]:
import numpy as np

# Define parameters
clip_length = 5  # Length of each clip in seconds
clip_length_samples = int(clip_length * fs)  # Convert clip length to samples

# Initialize lists to store clips and labels
clips = []
labels = []

# Iterate through each state
for state in lfp.keys():
    for segment in lfp[state]:
        # Calculate the number of clips in the segment
        num_clips = len(segment) // clip_length_samples

        # Cut the segment into smaller clips
        for i in range(num_clips):
            start_index = i * clip_length_samples
            end_index = start_index + clip_length_samples
            clip = segment[start_index:end_index]

            # Append the clip and its corresponding label to the lists
            clips.append(clip)
            labels.append(state)

# Convert lists to numpy arrays
clips = np.array(clips)
labels = np.array(labels)

# Print the shape of the resulting arrays
print("Shape of clips array:", clips.shape)
print("Shape of labels array:", labels.shape)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')